In [1]:
from PIL import Image, ImageOps
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import os, shutil
import time
import scipy
from scipy import ndimage
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import cv2
from keras import models
from keras import layers
from sklearn.utils import class_weight

# Now lets try and to make the results better

In [2]:
# Create a data generator
datagen2 = ImageDataGenerator(
    rescale=1./255, # normalize pixel values
    rotation_range=20, # randomly rotate images in the range
    zoom_range=0.2, # randomly zoom image 
    width_shift_range=0.2, # randomly shift images horizontally
    height_shift_range=0.2, # randomly shift images vertically 
    horizontal_flip=True) # randomly flip images

In [3]:
# Use flow_from_directory to read images from folders
train_generator2 = datagen2.flow_from_directory(
        'CellData/chest_xray/train', # point to the parent directory
        target_size=(299, 299), # resize images to 299x299
        batch_size=32,
        class_mode='binary') # binary for two classes

# Similarly create validation and test generators
val_generator2 = datagen2.flow_from_directory(
        'CellData/chest_xray/val',
        target_size=(299, 299),
        batch_size=32,
        class_mode='binary')

test_generator2 = datagen2.flow_from_directory(
        'CellData/chest_xray/test',
        target_size=(299, 299),
        batch_size=32,
        class_mode='binary')


Found 4098 images belonging to 2 classes.
Found 879 images belonging to 2 classes.
Found 879 images belonging to 2 classes.


In [4]:
model2 = models.Sequential()
model2.add(layers.Conv2D(32, (5, 5), activation='relu',
                        input_shape=(299 , 299,  3)))
model2.add(layers.MaxPooling2D((2, 2)))

model2.add(layers.Conv2D(32, (4, 4), activation='relu'))
model2.add(layers.MaxPooling2D((2, 2)))

model2.add(layers.Conv2D(64, (3, 3), activation='relu'))
model2.add(layers.MaxPooling2D((2, 2)))

model2.add(layers.Flatten())
model2.add(layers.Dense(64, activation='relu'))
model2.add(layers.Dense(1, activation='sigmoid'))

model2.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['acc'])

Metal device set to: Apple M2 Pro


2023-07-12 15:30:06.653852: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-07-12 15:30:06.654011: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
epochs = 30

steps_per_epoch2 = np.ceil(train_generator2.samples / train_generator2.batch_size)
validation_steps2 = np.ceil(val_generator2.samples / val_generator2.batch_size)

history = model2.fit(
    train_generator2,
    steps_per_epoch=int(steps_per_epoch2),
    epochs=epochs,
    validation_data=val_generator2,
    validation_steps=int(validation_steps2))



Epoch 1/30


2023-07-12 15:30:21.018637: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-07-12 15:30:21.262542: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


129/129 [==============================] - ETA: 0s - loss: 0.4339 - acc: 0.7897

2023-07-12 15:31:33.740962: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


129/129 [==============================] - 87s 631ms/step - loss: 0.4339 - acc: 0.7897 - val_loss: 0.4779 - val_acc: 0.7645
Epoch 2/30
129/129 [==============================] - 82s 634ms/step - loss: 0.3595 - acc: 0.8477 - val_loss: 0.3181 - val_acc: 0.8749
Epoch 3/30
129/129 [==============================] - 86s 643ms/step - loss: 0.3212 - acc: 0.8580 - val_loss: 0.2497 - val_acc: 0.8931
Epoch 4/30
129/129 [==============================] - 76s 586ms/step - loss: 0.3181 - acc: 0.8633 - val_loss: 0.2943 - val_acc: 0.8771
Epoch 5/30
129/129 [==============================] - 73s 564ms/step - loss: 0.2769 - acc: 0.8821 - val_loss: 0.2885 - val_acc: 0.8680
Epoch 6/30
129/129 [==============================] - 73s 561ms/step - loss: 0.2849 - acc: 0.8751 - val_loss: 0.2168 - val_acc: 0.9010
Epoch 7/30
129/129 [==============================] - 73s 566ms/step - loss: 0.2746 - acc: 0.8816 - val_loss: 0.2538 - val_acc: 0.8851
Epoch 8/30
129/129 [==============================] - 72s 554ms/st

In [9]:
results_train = model2.evaluate(train_generator2)
results_test = model2.evaluate(val_generator2)
print('Training set performance:', results_train)
print('Validation set performance:', results_test)


28/28 [==============================] - 13s 447ms/step - loss: 0.1784 - acc: 0.9306
Training set performance: [0.18181194365024567, 0.9294778108596802]
Validation set performance: [0.17840376496315002, 0.93060302734375]


## 93 percent is not bad